In [13]:

import time
import os
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import shutil

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split

In [14]:

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 256
data_transforms = {
    "train": transforms.Compose(
        [
            # transforms.RandomResizedCrop(224),     # uncomment for data augmentation
            # transforms.RandomHorizontalFlip(),     # uncomment for data augmentation
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            # Normalize input channels using mean values and standard deviations of ImageNet.
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "test": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

data_dir = "dataset/"
# data_dir = "data/hymenoptera_data"
image_datasets = {
    x if x == "train" else "test": datasets.ImageFolder(
        os.path.join(data_dir, x), data_transforms[x]
    )
    for x in ["train", "test"]
}

In [15]:
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "test"]}
class_names = image_datasets["train"].classes

In [16]:
# Initialize dataloader
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in ["train", "test"]
}

In [17]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 10000.0  # Large arbitrary number
    best_acc_train = 0.0
    best_loss_train = 10000.0  # Large arbitrary number
    
    print("Training started:")

    for epoch in range(num_epochs):

        # Each epoch has a training and validation phase
        for phase in ["train", "test"]:
            if phase == "train":
                # Set model to training mode
                model.train()
            else:
                # Set model to evaluate mode
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = dataset_sizes[phase] // batch_size
            it = 0
            for inputs, labels in dataloaders[phase]:
                since_batch = time.time()
                batch_size_ = len(inputs)
                inputs = inputs.to(device)
                labels = labels.to(device)
#                 print(labels)
                optimizer.zero_grad()

                # Track/compute gradient and make an optimization step only when training
                with torch.set_grad_enabled(phase == "train"):
#                     For neurons more than 1 in last layer
                    outputs = model(inputs)
#                     print(outputs)
                    _, preds = torch.max(outputs, 1)
                #     For one neuron in last layer
#                     outputs = model(inputs)
#                     preds=outputs
#                     print(preds)
#                     print(outputs)
                    loss = criterion(outputs, labels)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Print iteration results
                running_loss += loss.item() * batch_size_
                batch_corrects = torch.sum(preds == labels.data).item()
                running_corrects += batch_corrects
                print(
                    "Phase: {} Epoch: {}/{} Iter: {}/{} Batch time: {:.4f}".format(
                        phase,
                        epoch + 1,
                        num_epochs,
                        it + 1,
                        n_batches + 1,
                        time.time() - since_batch,
                    ),
                    end="\r",
                    flush=True,
                )
                it += 1

            # Print epoch results
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print(
                "Phase: {} Epoch: {}/{} Loss: {:.4f} Acc: {:.4f}        ".format(
                    "train" if phase == "train" else "test  ",
                    epoch + 1,
                    num_epochs,
                    epoch_loss,
                    epoch_acc,
                )
            )

            # Check if this is the best model wrt previous epochs
            if phase == "test" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(copy.deepcopy(model.state_dict()),'dogsCatModel.pth')
            if phase == "test" and epoch_loss < best_loss:
                best_loss = epoch_loss
            if phase == "train" and epoch_acc > best_acc_train:
                best_acc_train = epoch_acc
            if phase == "train" and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss

            # Update learning rate
            if phase == "train":
                scheduler.step()

    # Print final results
    model.load_state_dict(best_model_wts)
    time_elapsed = time.time() - since
    print(
        "Training completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60)
    )
    print("Best test loss: {:.4f} | Best test accuracy: {:.4f}".format(best_loss, best_acc))
    return model

In [18]:
step = 0.0004   
gamma_lr_scheduler = 0.1

class ClassicalTransferLearningModel(nn.Module):
    def __init__(self,num_of_features):
        super().__init__()
        self.fc1 = nn.Linear(num_of_features,2)
#         self.sigmoid = nn.Sigmoid()
        self.softmax= nn.Softmax(dim=1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [20]:
model_classical = torchvision.models.resnet152(pretrained=True)

for param in model_classical.parameters():
    param.requires_grad = False

num_ftrs = model_classical.fc.in_features
model_classical.fc = ClassicalTransferLearningModel(num_ftrs)
model_classical = model_classical.to(device)

criterion_classical = nn.CrossEntropyLoss()
optimizer_classical = optim.Adam(model_classical.fc.parameters(),lr=step)
exp_lr_scheduler_classical = lr_scheduler.StepLR(
    optimizer_classical, step_size=10, gamma=gamma_lr_scheduler
)

In [21]:
model_classical = train_model(model_classical,criterion_classical,optimizer_classical,exp_lr_scheduler_classical,num_epochs=5)

Training started:
Phase: train Epoch: 1/5 Loss: 0.4868 Acc: 0.9093        
Phase: test   Epoch: 1/5 Loss: 0.3810 Acc: 0.9815        
Phase: train Epoch: 2/5 Loss: 0.3685 Acc: 0.9782        
Phase: test   Epoch: 2/5 Loss: 0.3547 Acc: 0.9880        
Phase: train Epoch: 3/5 Loss: 0.3534 Acc: 0.9816        
Phase: test   Epoch: 3/5 Loss: 0.3474 Acc: 0.9840        
Phase: train Epoch: 4/5 Loss: 0.3469 Acc: 0.9835        
Phase: test   Epoch: 4/5 Loss: 0.3404 Acc: 0.9885        
Phase: train Epoch: 5/5 Loss: 0.3421 Acc: 0.9848        
Phase: test   Epoch: 5/5 Loss: 0.3375 Acc: 0.9885        
Training completed in 210m 13s
Best test loss: 0.3375 | Best test accuracy: 0.9885


# Testing the dogsAndCats.pth model

In [47]:
def test_model(model):
    since = time.time()
    print("Validation started:")

    model.eval()
    
    
    running_corrects_test=0

    batch_corrects_test=0
    phase = "test"

    n_batches = dataset_sizes[phase]
    it=0

    for inputs,labels in dataloaders[phase]:
        batch_size_ = len(inputs)
        inputs = inputs.to(device)
        labels = labels.to(device)
#         pred_cln = predict_from_logits(model(inputs))
        
        outputs = model(inputs)
        _, pred_cln = torch.max(outputs, 1)
        
        batch_corrects_test = torch.sum(pred_cln == labels.data).item()
        
        # running_corrects += batch_corrects
        running_corrects_test += batch_corrects_test
        
    # final_acc = running_corrects / dataset_sizes[phase]
    final_acc_test = running_corrects_test / dataset_sizes[phase]
    
    # print("Final Accuracy(original model): ",final_acc,final_acc_test)
    time_elapsed = time.time() - since
    print(
        "Testing completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60)
    )


    print("Final Accuracy(original model): ",final_acc_test)
    return final_acc_test

In [48]:
def load_model(model,path):
    print(path)
    model.load_state_dict(torch.load(path))  
    model.eval()
    return model

def classicalModel():
    model_classical = torchvision.models.resnet152(pretrained=True)
    
    for param in model_classical.parameters():
        param.requires_grad = False
        
    num_ftrs = model_classical.fc.in_features
    model_classical.fc = ClassicalTransferLearningModel(num_ftrs)
    model_classical = model_classical.to(device)
#     print(model_classical)
    return model_classical

In [49]:
path1 = "C:\\Users\\reekm\\Documents\\MachineLearningMathematics\\dogsCatModel.pth"
path =  "dogsCatModel.pth"

In [50]:
model = classicalModel()
model = load_model(model,path)
test_acc = test_model(model)
test_acc

C:\Users\reekm\Documents\MachineLearningMathematics\dogsCatModel.pth
Validation started:
Testing completed in 7m 46s
Final Accuracy(original model):  0.9885


0.9885